# UAS

## Crawling Portal Berita Antaranews

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import random

- request: Library ini digunakan untuk mengirim HTTP requests (seperti GET, POST, PUT, DELETE) dan menerima respons dari server. Dengan menggunakan Requests, Anda dapat mengakses dan mengambil data dari URL, mengirim data form, dan berinteraksi dengan API.

- beautifulSoup: Beautiful Soup adalah library Python yang digunakan untuk mengambil data dari HTML dan XML. Ini menyediakan cara yang mudah untuk menjelajahi, mencari, dan memanipulasi struktur data dokumen web. Beautiful Soup bekerja dengan parser HTML, membantu Anda mengekstrak informasi dari halaman web dengan lebih mudah.

- csv: Modul CSV di Python menyediakan fungsi untuk membaca dan menulis file CSV. Ini memungkinkan Anda untuk dengan mudah bekerja dengan data dalam format CSV, yang umumnya digunakan untuk menyimpan data tabel, terutama dalam konteks analisis data dan pertukaran data antar program.

- random: Library ini menyediakan fungsi-fungsi untuk menghasilkan angka acak. Dalam konteks web scraping atau pengujian, Anda mungkin menggunakan library ini untuk memilih elemen acak dari daftar atau untuk menyimulasikan perilaku yang tidak terduga.



In [ ]:
# masukkan url
response = requests.get("https://www.antaranews.com/")
# Isi teks dari respons HTTP yang diterima dari server web setelah melakukan permintaan GET.
soup = BeautifulSoup(response.text, 'html.parser')
# menemukan semua list yang berisi link kategori
first_page = soup.findAll('li',"dropdown mega-full menu-color1")

# menyimpan kategori
save_categori = []
for links in first_page:
  categori = links.find('a').get('href')
  save_categori.append(categori)
# save_categori

# categori yang akan dicari terdapat pada indeks 1,3,6
categori_search = [save_categori[3],save_categori[6],save_categori[9]]
categori_search

['https://www.antaranews.com/ekonomi',
 'https://www.antaranews.com/olahraga',
 'https://www.antaranews.com/hiburan']

In [ ]:
# Inisialisasi list untuk menyimpan data berita
datas = []

# Iterasi melalui halaman berita
for ipages in range(1, 3):

    # Iterasi melalui setiap kategori berita
    for beritas in categori_search:
        # Permintaan untuk halaman berita
        response_berita = requests.get(beritas + "/" + str(ipages))
        namecategori = beritas.split("/")

        # Parsing halaman berita dengan BeautifulSoup
        soup_berita = BeautifulSoup(response_berita.text, 'html.parser')
        pages_berita = soup_berita.findAll('article', {'class': 'simple-post simple-big clearfix'})

        # Iterasi melalui setiap artikel dalam halaman berita
        for items in pages_berita:
            # Mendapatkan link artikel
            get_link_in = items.find("a").get("href")

            # Request untuk halaman artikel
            response_artikel = requests.get(get_link_in)
            soup_artikel = BeautifulSoup(response_artikel.text, 'html.parser')

            # Ekstraksi informasi dari halaman artikel
            judul = soup_artikel.find("h1", "post-title").text if soup_artikel.findAll("h1", "post-title") else ""
            label = namecategori[-1]
            date = soup_artikel.find("span", "article-date").text if soup_artikel.find("span", "article-date") else "Data tanggal tidak ditemukan"

            trash1 = ""
            cek_baca_juga = soup_artikel.findAll("span", "baca-juga")
            if cek_baca_juga:
                for bacas in cek_baca_juga:
                    text_trash = bacas.text
                    trash1 += text_trash + ' '

            artikels = soup_artikel.find_all('div', {'class': 'post-content clearfix'})
            artikel_content = artikels[0].text if artikels else ""
            artikel = artikel_content.replace("\n", " ").replace("\t", " ").replace("\r", " ").replace(trash1, "").replace("\xa0", "")

            author = soup_artikel.find("p", "text-muted small mt10").text.replace("\t\t", "") if soup_artikel.findAll("p", "text-muted small mt10") else ""

            # Menambahkan data artikel ke dalam list
            datas.append({'Tanggal': date, 'Penulis': author, 'Judul': judul, 'Artikel': artikel, 'Label': label})


In [ ]:
# Iterasi melalui halaman berita
for ipages in range(1, 3):
    # Mengecek apakah iterasi saat ini kurang dari 2
    if ipages < 2:
        # Menampilkan jumlah data yang telah berhasil di-crawling
        print(f'Data berhasil dicrawling sebanyak : {len(datas)}')

In [ ]:
# Menyimpan data dalam bentuk CSV
csv_filename = 'berita_politik_antaranews.csv'
# Membuka file CSV untuk ditulis ('w' mode) dengan encoding utf-8
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Menentukan kolom (fieldnames) untuk file CSV
    fieldnames = ['Tanggal', 'Penulis', 'Judul', 'Artikel', 'Label']
    # Membuat objek DictWriter untuk menulis data ke file CSV
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    # Menulis header (baris pertama) ke file CSV
    csv_writer.writeheader()
    # Iterasi melalui data yang akan disimpan (diasumsikan datas sudah didefinisikan sebelumnya)
    for entry in datas:
        # Menulis setiap baris data ke file CSV
        csv_writer.writerow(entry)
# Menampilkan pesan bahwa data telah disimpan dalam file CSV
print(f"Data telah disimpan dalam file {csv_filename}")

## Import library/package yang dibutuhkan (Memproses Data)

In [1]:
import pandas as pd
import re

- Pandas adalah library open-source yang ditulis dalam bahasa pemrograman Python untuk analisis data dan manipulasi data. Dikembangkan oleh Wes McKinney, Pandas menyediakan struktur data yang efisien dan mudah digunakan untuk memanipulasi dan menganalisis data numerik dan tabular. Pandas sangat populer di kalangan data scientist, analis data, dan pengembang perangkat lunak karena kemudahan penggunaannya dan dukungan yang luas dalam ekosistem Python

- re adalah library Python yang menyediakan dukungan untuk ekspresi reguler (regular expressions). Ekspresi reguler adalah urutan karakter yang membentuk pola pencarian. Modul re memungkinkan pencocokkan pola dengan string dan melakukan berbagai operasi seperti pencarian, pencocokan, dan manipulasi string berdasarkan pola tertentu.

## Load data

Memanggil dataset yang akan digunakan dan ditampilkan dalam bentuk dataframe

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/dennywr/cobaprosaindata/main/berita_antaranews_fhd.csv')

In [3]:
df = pd.DataFrame(data)
df.head(5)

,Tanggal,Penulis,Judul,Artikel,Label
0,"Selasa, 5 Desember 2023 08:09 WIB",Pewarta: Benardy FerdiansyahEditor: Ahmad Wija...,PGE sebut panas bumi merupakan energi terbaruk...,Ini dikarenakan panas bumi memiliki ketersedi...,ekonomi
1,"Selasa, 5 Desember 2023 08:07 WIB",Penerjemah: Citro AtmokoEditor: Ahmad WijayaCO...,Dolar menguat didorong kenaikan imbal hasil ob...,Dengan perkiraan pelonggaran sebesar 135 basi...,ekonomi
2,"Selasa, 5 Desember 2023 07:00 WIB",NaN,Sektor pariwisata bersiap sambut libur Natal 2023,Kementerian Pariwisata dan Ekonomi Kreatif...,ekonomi
3,"Selasa, 5 Desember 2023 06:51 WIB",Pewarta: SumarwotoEditor: Ahmad WijayaCOPYRIGH...,KAI: Jalur hilir Purwokerto-Cirebon sudah bisa...,Jalur hilir di titik yang terdampak longsor s...,ekonomi
4,"Selasa, 5 Desember 2023 06:48 WIB",Pewarta: SubagyoEditor: Ahmad WijayaCOPYRIGHT ...,Produsen AMDK komitmen atasi krisis air bersih...,Permasalahan air bersih sudah selayaknya menj...,ekonomi


In [4]:
df['Artikel'] = df['Artikel'].fillna('').astype(str)

## Preprocessing

Preprocessing merupakan serangkaian langkah atau tahapan yang dilakukan untuk membersihkan, merapihkan, dan mengubah data teks mentah menjadi bentuk yang lebih sesuai untuk analisis atau pemodelan. Proses ini penting karena data teks seringkali datang dalam bentuk yang tidak terstruktur dan memerlukan persiapan khusus sebelum dapat digunakan secara efektif

In [5]:
!pip install indoNLP
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.9 MB/s eta 0:00:00


In [6]:
import numpy as np # numpy
import re # regex
import string # string population
from nltk.tokenize import word_tokenize # tokenize
from nltk.corpus import stopwords # stopword
from indoNLP.preprocessing import replace_slang # slank word
from nltk.stem.porter import PorterStemmer # stemming
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.decomposition import PCA #PCA
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
df['Artikel'].isna().sum()

0

In [8]:
df['Artikel'].dropna(inplace=True)

In [9]:
df['Label'].isna().sum()

0

In [10]:
df['Label'].dropna(inplace=True)

In [11]:
df['Artikel'].isna().sum()

0

In [12]:
df['Label'].isna().sum()

0

In [13]:
class Prepocessing:
  def __init__(self):
    self.listStopword =  set(stopwords.words('indonesian'))
    # self.stemmer = PorterStemmer()
    self.stemmer = StemmerFactory().create_stemmer()

  def remove_emoji(self, string): #remove emoji
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

  def cleaning(self, document): # clean text
    # remove user mentions
    document = re.sub("@[A-Za-z0-9_]+"," ", document)
    # remove URLS
    document = re.sub(r'http\S+', ' ', document)
    # remove hashtags
    document = re.sub("#[A-Za-z0-9_]+","", document)
    # remove emoji's
    document = self.remove_emoji(document)
    # remove punctuation
    document = re.sub("[^0-9A-Za-z ]", "" , document)
    # remove number
    document = re.sub(r'\b\d+\b', ' ', document)
    # remove double spaces
    document = document.replace('  '," ")
    return document.strip()

  def tokenize(self, text): #tokenize -> memisah kalimat
    return word_tokenize(text.translate(str.maketrans('', '', string.punctuation)).lower())

  def stopWord(self, text): #stopword -> menghapus kata hubung
    return [kata for kata in text if kata not in self.listStopword]

  def slank_word(self, text): #slank word -> mengganti kata yang tidak baku
    return [replace_slang(kata) for kata in text]

  def stemming(self, text): #stemming -> mengganti kata menjadi kata dasar
    return " ".join([self.stemmer.stem(kata) for kata in text])

In [14]:
preprocessing = Prepocessing()

- Cleaning

> Menghapus karakter khusus, tanda baca berlebihan, atau elemen-elemen tidak diinginkan lainnya (emoji, mention, url, hashtags, double space, dan punctuation) untuk membuat teks lebih bersih.

In [15]:
df['clean'] = df['Artikel'].apply(lambda x: preprocessing.cleaning(x))

- Tokenisasi

> Memisahkan sepotong teks menjadi unit yang lebih kecil yang disebut token. Di sini, token dapat berupa kata, karakter, atau subkata. Oleh karena itu, tokenisasi dapat secara luas diklasifikasikan menjadi 3 jenis - tokenisasi kata, karakter, dan subkata (karakter n-gram).

In [16]:
df['tokenize'] = df['clean'].apply(lambda x: preprocessing.tokenize(x))

- Penghapusan stopwords

> Stopwords adalah kata-kata umum yang sering muncul dalam suatu teks dan dianggap kurang bermakna karena keberadaannya yang umum. Dalam pemrosesan teks dan analisis teks, stopwords biasanya dihapus dari teks untuk meningkatkan kualitas analisis. Contoh stopwords dalam bahasa Indonesia meliputi kata-kata seperti "yang", "dan", "di", "dari", dan sebagainya.

In [17]:
df['stopword'] = df['tokenize'].apply(lambda x: preprocessing.stopWord(x))

- Slankwords Correction (Normalisasi Teks)

> Menggantikan kata-kata singkatan atau istilah non-formal dengan bentuk standar atau formal.

In [18]:
df['slankword'] = df['stopword'].apply(lambda x: preprocessing.slank_word(x))

- Stemming

> Stemming adalah proses pengurangan kata ke bentuk dasarnya atau bentuk kata yang lebih sederhana dengan menghapus akhiran atau awalan. Tujuan dari stemming adalah untuk menghasilkan bentuk kata dasar yang dapat mewakili kelompok kata serupa, misalnya kata "berjalan" menjadi "jalan", dan kata "memakan" menjadi "makan"

In [19]:
# df['stem'] = df['slankword'].apply(lambda x: preprocessing.stemming(x))

- Menampilkan hasil preprocessing

In [20]:
# df[['Artikel', 'clean', 'tokenize', 'stopword', 'slankword', 'stem', 'Label']]
df[['Artikel', 'clean', 'tokenize', 'stopword', 'slankword', 'Label']]

,Artikel,clean,tokenize,stopword,slankword,Label
0,Ini dikarenakan panas bumi memiliki ketersedi...,Ini dikarenakan panas bumi memiliki ketersedia...,"[ini, dikarenakan, panas, bumi, memiliki, kete...","[panas, bumi, memiliki, ketersediaan, terbaik,...","[panas, bumi, memiliki, ketersediaan, terbaik,...",ekonomi
1,Dengan perkiraan pelonggaran sebesar 135 basi...,Dengan perkiraan pelonggaran sebesar basis po...,"[dengan, perkiraan, pelonggaran, sebesar, basi...","[perkiraan, pelonggaran, basis, poin, desember...","[perkiraan, pelonggaran, basis, poin, desember...",ekonomi
2,Kementerian Pariwisata dan Ekonomi Kreatif...,Kementerian Pariwisata dan Ekonomi Kreatif Kem...,"[kementerian, pariwisata, dan, ekonomi, kreati...","[kementerian, pariwisata, ekonomi, kreatif, ke...","[kementerian, pariwisata, ekonomi, kreatif, ke...",ekonomi
3,Jalur hilir di titik yang terdampak longsor s...,Jalur hilir di titik yang terdampak longsor su...,"[jalur, hilir, di, titik, yang, terdampak, lon...","[jalur, hilir, titik, terdampak, longsor, ka, ...","[jalur, hilir, titik, terdampak, longsor, kak,...",ekonomi
4,Permasalahan air bersih sudah selayaknya menj...,Permasalahan air bersih sudah selayaknya menja...,"[permasalahan, air, bersih, sudah, selayaknya,...","[permasalahan, air, bersih, selayaknya, tanggu...","[permasalahan, air, bersih, selayaknya, tanggu...",ekonomi
...,...,...,...,...,...,...
1075,Jakarta (ANTARA) - Grup idola perempuan N...,Jakarta ANTARA Grup idola perempuan NewJeans d...,"[jakarta, antara, grup, idola, perempuan, newj...","[jakarta, grup, idola, perempuan, newjeans, sa...","[jakarta, grup, idola, perempuan, newjeans, sa...",hiburan
1076,Jakarta (ANTARA) - Film horor “Pemukiman ...,Jakarta ANTARA Film horor Pemukiman Setan dari...,"[jakarta, antara, film, horor, pemukiman, seta...","[jakarta, film, horor, pemukiman, setan, rumah...","[jakarta, film, horor, pemukiman, setan, rumah...",hiburan
1077,"Jakarta (ANTARA) - Aktor ""Grey's Anatomy""...",Jakarta ANTARA Aktor Greys Anatomy Patrick Dem...,"[jakarta, antara, aktor, greys, anatomy, patri...","[jakarta, aktor, greys, anatomy, patrick, demp...","[jakarta, aktor, greys, anatomy, patrick, demp...",hiburan
1078,,,[],[],[],hiburan


In [21]:
# X, y = df['Artikel'], df['Label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) adalah metode yang digunakan mengevaluasi seberapa penting sebuah kata bagi sebuah dokumen dalam kumpulan dokumen dengan cara mengalikan berapa kali sebuah kata muncul dalam sebuah dokumen (Term Frequency) dengan frekuensi dokumen terbalik (Inverse Document Frequency) dari kata di seluruh set dokumen.




### Term Frequency (TF)

$\text{TF}(t, d) = \frac{\text{jumlah kemunculan term } t \text{ dalam dokumen } d}{\text{total jumlah term dalam dokumen } d}
$

### Inverse Document Frequency (IDF)

$\text{IDF}(t, D) = \log\left(\frac{\text{total jumlah dokumen dalam korpus } |D|}{\text{jumlah dokumen yang mengandung term } t + 1}\right)
$

### Term Frequency-Inverse Document Frequency (TF-IDF)

$\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)$


Penjelasan:

- t adalah term (kata kunci),
- d adalah dokumen,
- D adalah korpus (kumpulan dokumen),
- ∣D∣ adalah total jumlah dokumen dalam korpus.

In [22]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [23]:
finalResult = pd.DataFrame(df, columns=["Artikel"])
label = pd.DataFrame(df, columns=["Label"])
# finalResult

> Membuat matriks TF-IDF dari teks dalam kolom 'Join Words' dari dataframe finalResult. Pertama, dilakukan impor modul stopwords dari NLTK untuk mendapatkan daftar stopwords dalam bahasa Indonesia. Selanjutnya, dibuat tokenizer dengan RegexpTokenizer untuk memproses teks. Kemudian, menggunakan TfidfVectorizer dari scikit-learn, teks diubah menjadi representasi vektor TF-IDF. Proses ini mencakup pengonversian huruf menjadi huruf kecil, penggunaan stopwords, dan penerapan tokenizer. Matriks TF-IDF yang dihasilkan, disimpan dalam variabel tfidf_matrix

In [24]:
stopwords = stopwords.words('indonesian')

tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)


tfidf_matrix = vectorizer.fit_transform(finalResult["Artikel"])
tfidf_terms = vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


### Vector Space Model (VSM)

> (Vector Space Model atau VSM) adalah representasi matematis dari dokumen-dokumen dalam suatu korpus atau koleksi teks. Dalam VSM, setiap dokumen direpresentasikan sebagai vektor dalam ruang vektor yang memiliki dimensi sebanding dengan jumlah kata unik dalam seluruh korpus. Setiap dimensi vektor mewakili kata tertentu, dan nilai dalam dimensi tersebut dapat mencerminkan berbagai informasi, seperti frekuensi kemunculan kata atau skor TF-IDF

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
# Membuat objek CountVectorizer
coun_vect = CountVectorizer(stop_words=stopwords)
count_matrix = coun_vect.fit_transform(finalResult["Artikel"])
count_array = count_matrix.toarray()
# Membuat DataFrame dari array dengan kolom berdasarkan term dalam teks
df = pd.DataFrame(data=count_array,columns = coun_vect.vocabulary_.keys())
df

,panas,bumi,memiliki,ketersediaan,terbaik,energi,terbarukan,dikontroljakarta,direktur,utama,...,piero,taruffi,2022penerjemah,luffy,bertualang,piracy,menjelajahi,bajak,hreeloita,shanti
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1076,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# menampilkan bobot dari masing-masing term
vsc = pd.DataFrame(data=tfidf_matrix.toarray(),columns = vectorizer.vocabulary_.keys())
print("\nTF-IDF Vectorizer\n")
vsc


TF-IDF Vectorizer



,panas,bumi,memiliki,ketersediaan,terbaik,energi,terbarukan,dikontroljakarta,direktur,utama,...,piero,taruffi,2022penerjemah,luffy,bertualang,piracy,menjelajahi,bajak,hreeloita,shanti
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.053448,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.027928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1076,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1077,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1078,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Klasifikasi Berita

> Klasifikasi berita adalah suatu bentuk tugas dalam bidang pemrosesan bahasa alami (Natural Language Processing) dan machine learning, di mana tujuan utamanya adalah untuk menentukan kategori atau label yang sesuai untuk suatu teks berdasarkan isinya

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

### Split Dataset

In [35]:
# Label
X = tfidf_matrix
y = label

# Bagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

TypeError: ignored

### Reduksi Dimensi (PCA)

In [29]:
n_components = 200
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train.toarray())
X_test_pca = pca.transform(X_test.toarray())


scaler = MinMaxScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

### Support Vector Machine (SVM)

In [30]:
# Buat model SVM
svm_model = SVC(kernel='rbf', C=1, gamma=0.1)

# Latih model pada data latih
svm_model.fit(X_train_pca, y_train)

# Lakukan prediksi pada data uji
y_pred_svm = svm_model.predict(X_test_pca)

# Evaluasi performa model SVM
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svm, zero_division=1))


SVM Accuracy: 0.9212962962962963

SVM Classification Report:
               precision    recall  f1-score   support

     ekonomi       0.98      0.83      0.90        71
     hiburan       0.83      0.99      0.90        77
    olahraga       1.00      0.94      0.97        68

    accuracy                           0.92       216
   macro avg       0.94      0.92      0.92       216
weighted avg       0.93      0.92      0.92       216



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Naive Bayes

In [31]:
from sklearn.naive_bayes import MultinomialNB

# Buat model Naive Bayes
naive_bayes_model = MultinomialNB()

# Latih model pada data latih
naive_bayes_model.fit(X_train_pca, y_train)

# Lakukan prediksi pada data uji
y_pred_nb = naive_bayes_model.predict(X_test_pca)

# Evaluasi performa model Naive Bayes
print("\nNaive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("\nNaive Bayes Classification Report:\n", classification_report(y_test, y_pred_nb, zero_division=1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Naive Bayes Accuracy: 0.9675925925925926

Naive Bayes Classification Report:
               precision    recall  f1-score   support

     ekonomi       0.91      1.00      0.95        71
     hiburan       1.00      0.97      0.99        77
    olahraga       1.00      0.93      0.96        68

    accuracy                           0.97       216
   macro avg       0.97      0.97      0.97       216
weighted avg       0.97      0.97      0.97       216



### K-Nearest Neighbor (KNN)

In [32]:
from sklearn.neighbors import KNeighborsClassifier

# Buat model KNN
knn_model = KNeighborsClassifier(n_neighbors=5)

# Latih model pada data latih
knn_model.fit(X_train_pca, y_train)

# Lakukan prediksi pada data uji
y_pred_knn = knn_model.predict(X_test_pca)

# Evaluasi performa model KNN
print("\nKNN Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nKNN Classification Report:\n", classification_report(y_test, y_pred_knn, zero_division=1))

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



KNN Accuracy: 0.5416666666666666

KNN Classification Report:
               precision    recall  f1-score   support

     ekonomi       1.00      0.28      0.44        71
     hiburan       0.51      0.66      0.58        77
    olahraga       0.48      0.68      0.56        68

    accuracy                           0.54       216
   macro avg       0.66      0.54      0.53       216
weighted avg       0.66      0.54      0.53       216



In [33]:
pip install joblib

In [34]:
import joblib

# Menyimpan model ke file
joblib.dump(knn_model, 'knn_model.pkl')

# Memuat model dari file
loaded_knn_model = joblib.load('knn_model.pkl')

from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming tfidf_vectorizer is the TfidfVectorizer used during training
new_data = "Bank Indonesia (BI) mencatat cadangan devisa Indonesia pada akhir November 2023 naik dibandingkan akhir Oktober 2023 karena dipengaruhi sejumlah faktor."

# Transform the new data using the same vectorizer
# new_data_tfidf = vectorizer.transform(new_data)

# Predict using the loaded model
result = knn_model.predict(new_data.reshape(1, -1))
print(result)



AttributeError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import pickle
import numpy as np

# Misalkan X_train adalah data latih Anda dalam bentuk teks
X_train = ["Contoh data latih 1", "Contoh data latih 2", "Contoh data latih 3"]

# Latih TfidfVectorizer pada X_train
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)

# Buat model PCA dan lakukan transformasi PCA pada data latih
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_tfidf.toarray())

# Buat model KNN dan latih pada data latih
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_pca, y_train)

# Simpan model KNN ke file
with open('knn_model.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

# Misalkan X_new adalah data baru Anda
X_new = ["Bank Indonesia (BI) mencatat cadangan devisa Indonesia pada akhir November 2023 naik dibandingkan akhir Oktober 2023 karena dipengaruhi sejumlah faktor."]

# Transformasikan data baru ke representasi TF-IDF
X_new_tfidf = tfidf.transform(X_new)

# Lakukan transformasi PCA pada data baru
X_new_pca = pca.transform(X_new_tfidf.toarray())

# Muat model KNN dari file
with open('knn_model.pkl', 'rb') as file:
    loaded_knn = pickle.load(file)

# Gunakan model KNN yang dimuat untuk membuat prediksi
y_new_pred = loaded_knn.predict(X_new_pca)

print("Prediksi untuk data baru:", y_new_pred)
